In [1]:
import numpy as np
import scipy.io as sio 
import scipy.signal as sig
from matplotlib import pyplot as plt
%matplotlib inline
dat1 = sio.loadmat('/Users/soumilhooda/Documents/BCI/Data/BCICIV_4_mat/sub1_comp.mat',struct_as_record = True)
dat2 = sio.loadmat('/Users/soumilhooda/Documents/BCI/Data/BCICIV_4_mat/sub2_comp.mat',struct_as_record = True)
dat3 = sio.loadmat('/Users/soumilhooda/Documents/BCI/Data/BCICIV_4_mat/sub3_comp.mat',struct_as_record = True)


In [2]:
# PRE-PROCESSING BLOCK

train_data1 = dat1['train_data']
train_dg1 = dat1['train_dg']
samples1,channels1 = train_data1.shape
tt = 400 #Sampling time
sf = 1000 #Sampling frequency 
channel_data1 = np.transpose(train_data1,(1,0))
finger_data1 = np.transpose(train_dg1,(1,0))

train_data2 = dat2['train_data']
train_dg2 = dat2['train_dg']
samples2,channels2 = train_data2.shape
tt = 400 #Sampling time
sf = 1000 #Sampling frequency 
channel_data2 = np.transpose(train_data2,(1,0))
finger_data2 = np.transpose(train_dg2,(1,0))


train_data3 = dat3['train_data']
train_dg3 = dat3['train_dg']
samples3,channels3 = train_data3.shape
tt = 400 #Sampling time
sf = 1000 #Sampling frequency 
channel_data3 = np.transpose(train_data3,(1,0))
finger_data3 = np.transpose(train_dg3,(1,0))

def bandPass(signal,fl,fh):
    taps = 505
    band = np.array([fl,fh])
    h = sig.firwin(taps,band,fs=sf,pass_zero='bandpass')
    output = np.convolve(signal,h,mode='same')
    return output

def AM(signal,samples):
    cur = 0
    output = []
    for i in range(samples):
        if(i and i % 40 == 0):
            output.append(cur)
            cur = 0
        cur += signal[i]**2
    output.append(cur)
         
    return output

sub_band1 = [bandPass(x,1,60) for x in channel_data1]
gamma_band1 = [bandPass(x,60,100) for x in channel_data1]
fast_gamma_band1 = [bandPass(x,100,200) for x in channel_data1]

sub_band2 = [bandPass(x,1,60) for x in channel_data2]
gamma_band2 = [bandPass(x,60,100) for x in channel_data2]
fast_gamma_band2 = [bandPass(x,100,200) for x in channel_data2]

sub_band3 = [bandPass(x,1,60) for x in channel_data3]
gamma_band3 = [bandPass(x,60,100) for x in channel_data3]
fast_gamma_band3 = [bandPass(x,100,200) for x in channel_data3]


sub_band1 = np.array([AM(x,samples1) for x in sub_band1])
gamma_band1 = np.array([AM(x,samples1) for x in gamma_band1])
fast_gamma_band1 = np.array([AM(x,samples1) for x in fast_gamma_band1])

sub_band2 = np.array([AM(x,samples2) for x in sub_band2])
gamma_band2 = np.array([AM(x,samples2) for x in gamma_band2])
fast_gamma_band2 = np.array([AM(x,samples2) for x in fast_gamma_band2])

sub_band3 = np.array([AM(x,samples3) for x in sub_band3])
gamma_band3 = np.array([AM(x,samples3) for x in gamma_band3])
fast_gamma_band3 = np.array([AM(x,samples3) for x in fast_gamma_band3])

finger_data1_ds = [None]*5
finger_data2_ds = [None]*5
finger_data3_ds = [None]*5
for i in range(5):
    finger_data1_ds[i] = [finger_data1[i][j] for j in range(0,samples1,40)]
    finger_data2_ds[i] = [finger_data2[i][j] for j in range(0,samples2,40)]
    finger_data3_ds[i] = [finger_data3[i][j] for j in range(0,samples3,40)]


dsamples1 = len(finger_data1_ds[0])
dsamples2 = len(finger_data2_ds[0])
dsamples3 = len(finger_data3_ds[0])

states1 = [None]*dsamples1
states2 = [None]*dsamples2
states3 = [None]*dsamples3

threshold_1,threshold_2  = 2.0,1.0

for i in range(dsamples1):
    flex,rest = 0,0
    for j in range(5):
        if finger_data1_ds[j][i] >= threshold_1:
            states1[i] = j + 1
            flex += 1
        elif finger_data1_ds[j][i] < threshold_2:
            rest += 1
    if states1[i] == None:
        if rest:
            states1[i] = 0


for i in range(dsamples2):
    flex,rest = 0,0
    for j in range(5):
        if finger_data2_ds[j][i] >= threshold_1:
            states2[i] = j + 1
            flex += 1
        elif finger_data2_ds[j][i] < threshold_2:
            rest += 1
    if states2[i] == None:
        if rest:
            states2[i] = 0



for i in range(dsamples3):
    flex,rest = 0,0
    for j in range(5):
        if finger_data3_ds[j][i] >= threshold_1:
            states3[i] = j + 1
            flex += 1
        elif finger_data3_ds[j][i] < threshold_2:
            rest += 1
    if states3[i] == None:
        if rest:
            states3[i] = 0




In [3]:
Sub1_SubBand = sub_band1
Sub1_GammaBand = gamma_band1
Sub1_FastGammaBand = fast_gamma_band1
Sub1_FingerState = states1

Sub2_SubBand = sub_band2
Sub2_GammaBand = gamma_band2
Sub2_FastGammaBand = fast_gamma_band2
Sub2_FingerState = states2

Sub3_SubBand = sub_band3
Sub3_GammaBand = gamma_band3
Sub3_FastGammaBand = fast_gamma_band3
Sub3_FingerState = states1

Training on just Subject 2 Gamma band at first.

In [4]:
import numpy 

Sub2_FingerState = numpy.asarray(Sub2_FingerState)
Sub2_FingerState = Sub2_FingerState.reshape(len(Sub2_FingerState), 1)

from sklearn.preprocessing import OneHotEncoder

Fingers_Subject2_OneHot = OneHotEncoder(sparse=False).fit_transform(Sub2_FingerState)
print(Fingers_Subject2_OneHot)

[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]


In [5]:
from sklearn.preprocessing import Normalizer

Sub2_GammaBand = Normalizer().fit(Sub2_GammaBand).transform(Sub2_GammaBand)
print(Sub2_GammaBand)

Sub2_GammaBand = numpy.array(Sub2_GammaBand)
Sub2_GammaBand = Sub2_GammaBand.T
print(Sub2_GammaBand.shape)
Sub2_GammaBand = Sub2_GammaBand.reshape(len(Sub2_GammaBand),1,len(Sub2_GammaBand[0]))
print(Sub2_GammaBand.shape)



[[0.00281701 0.00491227 0.01126519 ... 0.0070843  0.01215409 0.01015238]
 [0.00209951 0.00975184 0.00242371 ... 0.00832406 0.00349465 0.03525792]
 [0.00504946 0.0050233  0.01047237 ... 0.00438427 0.00439439 0.00756943]
 ...
 [0.01126505 0.00894738 0.01334184 ... 0.01278444 0.00802668 0.01500387]
 [0.05936403 0.00494066 0.00175576 ... 0.00815658 0.00519528 0.01643311]
 [0.01092104 0.00966892 0.01511716 ... 0.01297696 0.0095079  0.01364453]]
(10000, 48)
(10000, 1, 48)


In [95]:
from sklearn.preprocessing import Normalizer

Sub2_FastGammaBand = Normalizer().fit(Sub2_FastGammaBand).transform(Sub2_FastGammaBand)
print(Sub2_FastGammaBand)

Sub2_FastGammaBand = numpy.array(Sub2_FastGammaBand)
Sub2_FastGammaBand = Sub2_FastGammaBand.T
print(Sub2_FastGammaBand.shape)
# Sub2_FastGammaBand = Sub2_FastGammaBand.reshape(
#     len(Sub2_FastGammaBand), len(Sub2_FastGammaBand[0]), 1)
# print(Sub2_FastGammaBand.shape)


[[0.01669842 0.01208196 0.00798052 ... 0.00840764 0.01497134 0.02912255]
 [0.00528902 0.00880696 0.00477819 ... 0.01994905 0.01428287 0.06911929]
 [0.00557161 0.00458561 0.00441833 ... 0.00834356 0.00471025 0.0074125 ]
 ...
 [0.05699484 0.009701   0.0083897  ... 0.01286451 0.00891634 0.01621813]
 [0.28559785 0.00548481 0.00678735 ... 0.00546382 0.00833229 0.024755  ]
 [0.00885882 0.0089432  0.00808341 ... 0.00949466 0.00838125 0.01396224]]
(10000, 48)


In [9]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2

model = keras.Sequential()
model.add(layers.LSTM(32,  input_shape=(1,48), activation = "relu"))
model.add(layers.Dense(6, activation='softmax'))

opt = keras.optimizers.Adam(learning_rate=0.005)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['mse'])
model.summary()

history = model.fit(Sub2_GammaBand, Fingers_Subject2_OneHot, batch_size = 1, epochs=1, validation_split=0.2)

predictions = model.predict(Sub2_GammaBand)
print("Predicted values are: ", predictions)
np.savetxt('FirstModel_NorPred.txt',predictions)
Corrected_Pred = numpy.argmax(predictions, axis = 1)
print(Corrected_Pred)
np.savetxt('FirstModel_Pred.txt',Corrected_Pred)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 32)                10368     
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 198       
Total params: 10,566
Trainable params: 10,566
Non-trainable params: 0
_________________________________________________________________
Train on 8000 samples, validate on 2000 samples
8000/8000 [==============================] - 37s 5ms/sample - loss: 0.8891 - mse: 0.0645 - val_loss: 0.8271 - val_mse: 0.0601
Predicted values are:  [[0.977778   0.00384999 0.00419019 0.0080591  0.00271104 0.00341162]
 [0.88766164 0.02162556 0.02156598 0.03502733 0.01550683 0.01861268]
 [0.7423791  0.0524128  0.04964415 0.07346267 0.03777738 0.04432382]
 ...
 [0.8350162  0.03253068 0.03177135 0.04957049 0.02332411 0.02778724]
 [0.79722536 0.04065466 0.03901076

-----------------------------------------------------------------------------------------------------------------------------------------

In [58]:
from sklearn import svm, datasets
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

X_train, X_test, y_train, y_test = model_selection.train_test_split(Sub2_GammaBand, Sub2_FingerState, train_size=0.80, test_size=0.20, random_state=101)

rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train, y_train)
poly = svm.SVC(kernel='poly', degree=3, C=1).fit(X_train, y_train)

poly_pred = poly.predict(X_test)
rbf_pred = rbf.predict(X_test)

poly_accuracy = accuracy_score(y_test, poly_pred)
poly_f1 = f1_score(y_test, poly_pred, average='weighted')
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

rbf_accuracy = accuracy_score(y_test, rbf_pred)
rbf_f1 = f1_score(y_test, rbf_pred, average='weighted')
print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))

Accuracy (Polynomial Kernel):  78.05
F1 (Polynomial Kernel):  68.43
Accuracy (RBF Kernel):  78.05
F1 (RBF Kernel):  68.43


In [64]:
from sklearn import svm, datasets
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

X_train, X_test, y_train, y_test = model_selection.train_test_split(Sub2_FastGammaBand, Sub2_FingerState, train_size=0.80, test_size=0.20, random_state=101)

rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train, y_train)
poly = svm.SVC(kernel='poly', degree=3, C=1).fit(X_train, y_train)

poly_pred = poly.predict(X_test)
rbf_pred = rbf.predict(X_test)

poly_accuracy = accuracy_score(y_test, poly_pred)
poly_f1 = f1_score(y_test, poly_pred, average='weighted')
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

rbf_accuracy = accuracy_score(y_test, rbf_pred)
rbf_f1 = f1_score(y_test, rbf_pred, average='weighted')
print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))

Accuracy (Polynomial Kernel):  78.10
F1 (Polynomial Kernel):  70.17
Accuracy (RBF Kernel):  78.05
F1 (RBF Kernel):  68.43
